In [ ]:
#Required Libraries and package of python
import nltk
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize,word_tokenize
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import spatial
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import nltk 
from nltk.tokenize import LineTokenizer 
from nltk.stem import PorterStemmer     
tk = LineTokenizer()   
lm = WordNetLemmatizer() 
from nltk.corpus import wordnet as wn
import re
import string

In [ ]:
#Pretrained Word2Vec model. It was pretrained on Google News Corpus
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
#Stopwords
stop_words = set(stopwords.words('english'))
sw=stop_words

In [ ]:
#STS-2017 Dataset is used here.Here, we are taking the first sentence and second sentence into two list. 
fh=open('STS.input.track51.en-en.txt')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    sentences=line.split('\t')
    s1.append(sentences[0])
    s2.append(sentences[1]) 
print(c)

# Maximum word-level Similarity(MWL_S)

In [ ]:
file=open('resultwordcommean1.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    s1word=[]
    for word in ns1:
        for c in string.punctuation:
            word=word.replace(c,'')
        if word not in sw:
            s1word.append(lemmatizer.lemmatize(word))
    #print(s1word)
    s2word=[]
    ns2=s2[i].split(' ')
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            s2word.append(lemmatizer.lemmatize(word))
    #print(s2word)
    max_score=[]
    for word in s1word:
        if word in model.wv.vocab:
            score=[]
            for w in s2word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score.append(max(score))
    #print(max_score)
    
    max_score2=[]
    for word in s2word:
        if word in model.wv.vocab:
            score=[]
            for w in s1word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score2.append(max(score))
    #print(max_score2)
    
    count=0
    summ=0
    for val in max_score:
        summ+=val
        count+=1
    average=(summ*5)/count
    
    count1=0
    summ1=0
    for val in max_score2:
        summ1+=val
        count1+=1
    average1=(summ1*5)/count1
    
    aver=(average+average1)/2
    #print('i:',i+1,aver)
    a=str(round(aver,6))+'\n'
    file.writelines(a)
    i+=1
    
file.close()
data1=[]
data2=[]
f1=open('resultwordcommean1.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Average Feature Vector based Similarity(V_avgS):

In [ ]:
file=open('result1.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    sum1=np.zeros((300,), dtype="float32")
    for word in ns1:
        for c in string.punctuation:
                word=word.replace(c,"")
        #print(word)
        if word not in sw:
            if(word in model.wv.vocab): 
                sum1=np.add(sum1,model[word])
    average1=np.divide(sum1,len(ns1),dtype="float32")
            
    ns2=s2[i].split(' ')
    sum2=np.zeros((300,),dtype="float32")
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                sum2=np.add(sum2,model[word])
    average2=np.divide(sum2,len(ns2),dtype="float32")
    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    a=str(round(sim,6))+'\n'
    file.writelines(a)
    #print(i,'similarity',sim)
    i+=1 
file.close()

from sklearn.metrics import mean_squared_error

data1=[]
data2=[]
f1=open('result1.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Maximum word-level similarity of NP, VP (MWL_NVP_S)

In [ ]:
file=open('resultNP_VP_compare123.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word.strip())
    return sent

def result(s1,s2):
    s1word=[]
    s2word=[]
    for word in s1:
        if word not in sw:
            s1word.append(word)
    for word in s2:
        if word not in sw:
            s2word.append(word)
            
    #print(s1word,s2word)
    if len(s1word)==0 or len(s2word)==0:
        aver=0
    else:
        max_score=[]
        for word in s1word:
            score=[0]
            if word in model.wv.vocab:
                for w in s2word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score.append(max(score))
        #print(max_score)
    
        max_score2=[]
        for word in s2word:
            score=[0]
            if word in model.wv.vocab:
                for w in s1word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score2.append(max(score))
        #print(max_score2)
    
        average=sum(max_score)/(len(max_score))
    
        average1=sum(max_score2)/(len(max_score2))
    
        aver=(average+average1)/2
    return aver

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp).strip()
    sen1np=sen1np.split(' ')
    #print(sen1np, len(sen1np))
    sen1vp=listtostr(vpp).strip()
    sen1vp=sen1vp.split(' ')
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp).strip()
    sen2np=sen2np.split(' ')
    sen2vp=listtostr(vpp).strip()
    sen2vp=sen2vp.split(' ')
    
    avernp=result(sen1np,sen2np)

    avervp=result(sen1vp,sen2vp)
    
    av=avernp*4+avervp
    av=str(round(av,6))+'\n'
    #print('No:',i+1,'similarity:',av)
    file.writelines(av)
    
    i=i+1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP_compare123.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse=np.square(np.subtract(data2,data1)).mean()
print("MSE:",mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Phrase Structured-based Similarity(Feature Vector based (AFS_NVP_S))

In [ ]:
file=open('resultNP_VP12.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word)
    return vectorvalue(sent)

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp)
    sen1vp=listtostr(vpp)
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp)
    sen2vp=listtostr(vpp)
        
    if ((np.all(sen1np==0)) or (np.all(sen2np==0))):
            nppsim=0
    else:
        nppsim= 1 - spatial.distance.cosine(sen1np,sen2np)

    if ((np.all(sen1vp==0)) or (np.all(sen2vp==0))):
            vppsim=0
    else:
        vppsim=1 - spatial.distance.cosine(sen1vp,sen2vp)
    similarity=nppsim*4+vppsim
    #print("No:",i,similarity)
    sim=str(round(similarity,2))+'\n'
    file.writelines(sim)
    i+=1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP12.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Phrase structure based Sim_NVP

In [ ]:
stop_words = set(stopwords.words('english'))
sw=stop_words
#print(sw)
fh=open('STS.input.track51.en-en.txt')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    sentences=line.split('\t')
    s1.append(sentences[0])
    s2.append(sentences[1])    
    #print(s1,s2)
print(c)

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

file=open('resultNP.txt','w')
def wnb(s1,s2):
    from nltk.corpus import wordnet as wn
    w1=[]
    w2=[]
    
    words1 = word_tokenize(s1)
    for word in words1:
        if word not in sw:
            w1.append(word)
    #print("word1:",w1)
    
    words2= word_tokenize(s2)
    for word in words2:
        if word not in sw:
            w2.append(word)
    #print("word2:",w2)
    w11=[]
    w22=[]
    for w in w1:
        try:
            #print(wn.synsets(w)[0])
            w11.append(wn.synsets(w)[0])
        except:
            continue
    for w in w2:
        try:
            #print(wn.synsets(w)[0])
            w22.append(wn.synsets(w)[0])
        except:
            continue
    x=[]
    for w in w11:
        s=[]
        for se in w22:
            try:
                sim=w.path_similarity(se)
                #print(sim)
                if(sim==None):
                    s.append(0)
                else:
                    s.append(sim)
            except:
                continue
        if (len(s)>0):
            x.append(max(s))
        #print(x)
    if(len(x)>0):
        si=sum(x)/len(x)
    else:
        si=0
    return si

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word)
    return sent

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
   # print(ns1)
    npp,vpp=partitioning(ns1)
   # print(npp,vpp)
    sen1np=listtostr(npp)
    sen1vp=listtostr(vpp)
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp)
    sen2vp=listtostr(vpp)
        
    nppsim=float(wnb(sen1np,sen2np))
   # print("np:", nppsim)
    vppsim=float(wnb(sen1vp,sen2vp))
    #print("vp:", vppsim)
    
     
    if(nppsim==0 or vppsim==0):
        similarity=(nppsim+vppsim)*5
    elif(nppsim !=0 and vppsim !=0):
        similarity= nppsim*4+vppsim
    #print("No:",i+1,similarity)
    sim=str(round(similarity,2))+'\n'
    file.writelines(sim)
    i+=1
    
file.close()

from scipy.stats import pearsonr
from scipy.stats import spearmanr
import numpy as np

data1=[]
data2=[]
f1=open('resultNP.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse=np.square(np.subtract(data2,data1)).mean()
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Role-based Maximum similarity RM_Sim

In [ ]:
file=open('newNpVp.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listvalue(pp):
    pplist=[]
    if(len(pp)>0):
        j=0
        while(j<len(pp)):
            pplist.append(vectorvalue(pp[j]))
            j=j+1
    else:
        pplist.append(np.zeros((300,), dtype="float32"))
    return pplist

def score(nplist,np2list,vplist,vp2list,a):
    simlist=[]
    if(np.all(nplist)==0 and np.all(np2list)==0 and np.all(vplist)==0 and np.all(vp2list)==0):
        aa=0
    else:
        for value in nplist:
            npsim=[]
            for value2 in np2list:
                sim=(1 - spatial.distance.cosine(value,value2))*(5/a)
                if( np.isnan(sim)==1):
                    sim=0
                npsim.append(sim)
            #print('np:',max(npsim))
            simlist.append(max(npsim))
        
        for v in vplist:
            vpsim=[]
            for v2 in vp2list:
                sim=(1-spatial.distance.cosine(v,v2))*(5/a)
                if np.isnan(sim)==1:
                    sim=0
                vpsim.append(sim)
            #print('vp:',max(vpsim))
            simlist.append(max(vpsim))
        if(len(simlist)>0):    
            bb=max(simlist)
            p=0
            while(p<len(simlist)):
                if simlist[p]<bb/2:
                    simlist[p]=0
                p=p+1
        aa = sum(simlist)
    return aa

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    nplist=[]
    vplist=[]
    np2list=[]
    vp2list=[]
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    nplist=listvalue(npp)
    vplist=listvalue(vpp)
    a=len(npp)+len(vpp)
    
    ns2=s2[i]
    #print(ns2)
    npp2,vpp2=partitioning(ns2)
    #print(npp2,vpp2)
    np2list=listvalue(npp2)
    vp2list=listvalue(vpp2)
    b=len(npp2)+len(vpp2)
    
    if a>=b:
        sim=score(nplist,np2list,vplist,vp2list,a)
    else:
        sim=score(np2list,nplist,vp2list,vplist,b)
    #print(i+1,'similarity:',sim)
    sim=str(round(sim,2))+'\n'
    file.writelines(sim)
    i=i+1
    
file.close()

data1=[]
data2=[]
f2=open('STS.gs.track5.en-en.txt')
f1=open('newNpVp.txt')
#f2=open('newNpVp.txt')

def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

In [ ]:
#Average of Phrase Structured based measures
f1=open('resultNP_VP_compare12.txt')
f2=open('resultNP_VP12.txt')
f3=open('newNpVp.txt')
f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
data3=listvalue(f3)
data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data1)):
    data=(data1[i]+data2[i]+data3[i]+data4[i])/4
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of without phrase structured measures
data1=[]
data2=[]
data3=[]
f1=open('result1.txt')
f2=open('resultwordcommean1.txt')
f3=open('STS.gs.track5.en-en.txt')
file=open("combineword.txt",'w')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
data3=listvalue(f3)
#print(len(data1),len(data2),len(data3))
i=0
aver1=[]
while(i<len(data1)):
    data=(data1[i]+data2[i])/2
    aver1.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
corr,_=pearsonr(data3,aver1)
scor,_=spearmanr(data3,aver1)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

# This is the End of our Code

In [ ]:
#Average of Phrase Structured based measures without RM_Sim
f1=open('resultNP_VP_compare12.txt')
f2=open('resultNP_VP12.txt')
#f3=open('newNpVp.txt')
f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase3.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#data3=listvalue(f3)
data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data1)):
    data=(data1[i]+data2[i]+data4[i])/3
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures except RM_Sim
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase3.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of Phrase Structured based measures without phrase word-level similarity
#f1=open('resultNP_VP_compare12.txt')
f2=open('resultNP_VP12.txt')
f3=open('newNpVp.txt')
f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase3.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
#data1=listvalue(f1)
data2=listvalue(f2)
data3=listvalue(f3)
data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data5)):
    data=(data3[i]+data2[i]+data4[i])/3
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
print(len(aver))
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures except Phrase word-level
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase3.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)

i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of Phrase Structured based measures without phrase average vector
f1=open('resultNP_VP_compare12.txt')
#f2=open('resultNP_VP12.txt')
f3=open('newNpVp.txt')
f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase3.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
data3=listvalue(f3)
data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data5)):
    data=(data3[i]+data1[i]+data4[i])/3
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
print(len(aver))
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures except phrase average vector
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase3.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of Phrase Structured based measures without WN POS similarity
f1=open('resultNP_VP_compare12.txt')
f2=open('resultNP_VP12.txt')
f3=open('newNpVp.txt')
#f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase3.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
data3=listvalue(f3)
#data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data5)):
    data=(data3[i]+data2[i]+data1[i])/3
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
print(len(aver))
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures except phrase WN POS
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase3.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of all measures except maximum word-level similarity
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('result1.txt')
f2=open('combinephrase2.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)

d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of all measures except average feature vector
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('resultwordcommean1.txt')
f2=open('combinephrase2.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)

d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of Phrase Structured based measures without WN POS similarity and RM_sim
f1=open('resultNP_VP_compare12.txt')
f2=open('resultNP_VP12.txt')
#f3=open('newNpVp.txt')
#f4=open('resultNP.txt')
f5=open('STS.gs.track5.en-en.txt')
file=open('combinephrase3.txt','w')
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#data3=listvalue(f3)
#data4=listvalue(f4)
data5=listvalue(f5)
aver=[]
i=0
while(i<len(data1)):
    data=(data2[i]+data1[i])/2
    aver.append(data)
    data=str(round(data,1))+'\n'
    file.writelines(data)
    i=i+1
print(len(aver))
corr,_=pearsonr(data5,aver)
scor,_=spearmanr(data5,aver)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)
file.close()

In [ ]:
#Average of all measures except phrase WN POS and RM_sim
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase3.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
print(len(d1),len(d2),len(d3))
i=0
while i<len(d1):
    if d1[i]<d2[i]:
        d4.append((d1[i]+d2[i])/2)
    else:
        d4.append(d1[i])
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)

In [ ]:
#Average of all measures
d1=[]
d2=[]
d3=[]
d4=[]
f1=open('combineword.txt')
f2=open('combinephrase.txt')
f3=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
d1=listvalue(f1)
d2=listvalue(f2)
d3=listvalue(f3)
i=0
while i<len(d1):
    d4.append((d1[i]+d2[i])/2)
    i=i+1
corr,_=pearsonr(d3,d4)
scor,_=spearmanr(d3,d4)
print("Pearson's:",corr*100)
print("Spearman's:",scor*100)


In [ ]:
i=0
c=0
while i<len(d3):
    a= d3[i]-d4[i]
    b=d4[i]-d3[i]
    #if(-0.1<a<0.1):
    if((a > 2.5) or (b > 2.5)):
        print('NO:',i+1,'GS:', d3[i],'Our Method:', d4[i])
        c=c+1
    i=i+1
print('count:',c)

In [ ]:
import xlsxwriter
workbook   = xlsxwriter.Workbook('score_comparison.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

i=0
while i<250:
    worksheet1.write_number(i,0, d3[i])
    worksheet1.write_number(i, 1, d4[i])
    i=i+1

workbook.close()